# Plot the Near-Inertial Activity Forecast

To see how we got here, check the notebooks about [loading the data](./01_download_GFS_data.html), and about [running the slab-ocean model](./02_run_slab_model.html).

Details: https://willirath.github.io/nia-prediction-low-latitutdes

## Parameters

In [ ]:
# parameters

GFS_zarr_store = "tmp_GFS.zarr"
slab_zarr_store = "tmp_slab.zarr"

dask_kwargs = {"n_workers": 1, "threads_per_worker": 4, "memory_limit": 6e9}

buoy_locations = {
    "20N,38W": {"lat": 20.0, "lon": -38.0},
    "15N,38W": {"lat": 15.0, "lon": -38.0},
    "20.5N,23W": {"lat": 20.5, "lon": -23.0},
    "11.5N,23W": {"lat": 11.5, "lon": -23.0},
    "6S,10W": {"lat": -6.0, "lon": -10.0},
    "10S,10W": {"lat": -10.0 ,"lon": -10.0},
}

## Tech preamble

Import modules and spin up Dask cluster.

In [ ]:
import numpy as np
import xarray as xr
from dask.distributed import Client
import hvplot.xarray
import cmocean
import pandas as pd
import cartopy.crs as ccrs


import holoviews as hv

from functools import reduce
from operator import add

In [ ]:
client = Client(**dask_kwargs)
client

## Load the data

In [ ]:
ds_GFS = xr.open_zarr(GFS_zarr_store)
ds_slab = xr.open_zarr(slab_zarr_store)

## Find start of forecast period

In [ ]:
start_of_forecast = (~ds_GFS["is_forecast"].astype(bool)).sum().compute().data
start_of_forecast = ds_GFS["time"].data[start_of_forecast]
print(start_of_forecast)

## Remove steady state

**FIXME:** This should have been done in the slab-model run?

In [ ]:
ds_slab["u_slab"] -= ds_slab["u_slab"].mean("time")
ds_slab["v_slab"] -= ds_slab["v_slab"].mean("time")
ds_slab["umag_slab"] = (ds_slab["u_slab"] ** 2 + ds_slab["v_slab"] ** 2) ** 0.5

## Plot for buoy locations

In [ ]:
plots = []

for name, location in buoy_locations.items():
    buoy_ds = ds_slab.sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    buoy_ds["taux"] = ds_GFS["taux"].sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    buoy_ds["tauy"] = ds_GFS["tauy"].sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    
    plots.append(
        (
            (
                buoy_ds["taux"].hvplot.line(label="taux", width=400, height=200)
                * buoy_ds["tauy"].hvplot.line(label="tauy")
            ).opts(legend_cols=True, title=name)
            + (
                buoy_ds["u_slab"].hvplot.line(label="u", width=400, height=200)
                * buoy_ds["v_slab"].hvplot.line(label="v")
                * buoy_ds["umag_slab"].hvplot.line(label="umag")
                * hv.VLine(start_of_forecast)
            ).opts(legend_cols=True, title=name)
        )
    )

reduce(add, plots).cols(2)

## Plot overview

In [ ]:
slab_umag_max = ds_slab["umag_slab"].where(ds_GFS["is_forecast"]).max("time")
slab_umag_max.hvplot.quadmesh(
    x="lon", y="lat", z="umag_slab",
    clim=(0, 2),
    cmap=cmocean.cm.speed,
    width=800, height=500,
    hover=False,
    geo=True, coastline=True,
    crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree()
)

In [ ]:
SLP_mean_analysis = ds_GFS["SLP"].where(~ds_GFS["is_forecast"]).mean("time")
SLP_anomaly = ds_GFS["SLP"] - SLP_mean_analysis

In [ ]:
display(
    SLP_mean_analysis.hvplot.quadmesh(
        clim=(97_000, 103_000),
        cmap=cmocean.cm.thermal,
        width=800, height=300,
        geo=True, coastline=True,
        crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
        hover=False
    )
)

In [ ]:
plot_times = [
    (start_of_forecast + n * np.timedelta64(12, "h"))
    for n in range(0, 8)
]

plots = []

for plot_time in plot_times:

    plots.append(
        SLP_anomaly.sel(time=plot_time, method="nearest").hvplot.quadmesh(
            clim=(-1000, 1000),
            cmap=cmocean.cm.balance,
            width=800, height=300,
            geo=True, coastline=True,
            crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
            hover=False
        ).opts(title=f"SLP anomaly, {plot_time}")
    )
    
reduce(add, plots).cols(1)

---

In [ ]:
!echo "Finished: $(date -Ins)"

---
See https://github.com/willirath/nia-prediction-low-latitutdes for details.